In [1]:
import sys
sys.path.append('/home/mapdadmin/abraham/caltrans-data-exploration/')

In [2]:
from configparser import ConfigParser
import pandas as pd

from process_traffic_data import apply_custom_transformations

In [3]:
from omnisci_connector.omni_connect import OmnisciConnect
#import ibis

In [4]:
print("connect to omnisci")
OmnisciHandle = OmnisciConnect(config_path)
OmnisciHandle.start_connection()

connect to omnisci


NameError: name 'config_path' is not defined

In [ ]:
OmnisciHandle.con.get_tables()

In [ ]:
OmnisciHandle.con.get_table_details('caltrans_historical_2015_2019_5min')

In [ ]:
cols = "timestamp_, \
freeway"

In [ ]:
df = OmnisciHandle.con.select_ipc("select " + cols + " from caltrans_historical_2015_2019_5min limit 100")

In [ ]:
df.dtypes

In [24]:
df.dtypes

timestamp_    datetime64[ns]
dtype: object

In [ ]:
config_path = '/home/mapdadmin/abraham/ini_files/config.ini'
print("read configuration file %s" %config_path)
config = ConfigParser()
config.read(config_path)
print("Configuration file read.")


In [ ]:
# useful columns:
traffic_data_columns = ['timestamp_',
                        'station',
                        'district',
                        'freeway',
                        'direction',
                        'lane_type',
                        'station_length',
                        'samples',
                        'pct_observed',
                        'total_flow',
                        'occupancy',
                        'speed'
                        ]

traffic_meta_columns = ['ID',
                        'County',
                        'State_PM',
                        'Abs_PM',
                        'Latitude',
                        'Longitude',
                        'Lanes',
                        'Name']

In [ ]:

### metadata section ###
traffic_meta_path = config.get('Paths', 'meta_path')

# read in the traffic metadata to pandas:
df_traffic_metadata = pd.read_csv(traffic_meta_path, sep='\t', usecols=traffic_meta_columns).set_index('ID')
df_traffic_metadata = df_traffic_metadata.rename(str.lower, axis='columns')
print("traffic metadata file read.")





In [ ]:
### Traffic Section ###

# get the paths of relevant files for the data
csv_files = config.get('Paths', 'data_path')

In [ ]:

# send traffic metadata:
    # get file paths:
file_paths = utils.get_file_names(csv_files, extension=file_ext)
print("Number of traffic files found: ", len(file_paths))

# extract and traffic data in batches:
no_data_cols = list(range(len(traffic_data_columns)))
for i in range(0, len(file_paths), batch_limit):
#for i in range(0, batch_limit, batch_limit):
    df_batch = []
    for f in file_paths[i:i + batch_limit]:
        print("Processing file: ", f)
        temp = pd.read_csv(f, header=None, names=traffic_data_columns, usecols=no_data_cols)
        df_batch.append(temp)

    df_extracted_traffic = pd.concat(df_batch, ignore_index=True)

    df_extracted_traffic = df_extracted_traffic.drop('district', axis=1)

    df_extracted_traffic = df_extracted_traffic.join(df_traffic_metadata, on='station')

    try:
        df_transformed_traffic = apply_custom_transformations(df=df_extracted_traffic,
                                                              interest_col=interest_col,
                                                              threshold=threshold,
                                                              grouper=grouper)
    except ValueError as ex:
        print(ex)
        print("Skipping Batch starting at: ", i)

In [ ]:
01/01/2019	New Year's Day	Tuesday
01/21/2019	Martin Luther King, Jr. Day	Monday
02/18/2019	Washington's Birthday	Monday
05/27/2019	Memorial Day	Monday
07/04/2019	Independence Day	Thursday
09/02/2019	Labor Day	Monday
10/14/2019	Columbus Day	Monday
11/11/2019	Veterans Day (Observed)	Monday
11/28/2019	Thanksgiving Day	Thursday
11/29/2019	Day after Thanksgiving	Friday
12/25/2019	Christmas Day	Wednesday